## Challenge: Exercise 4

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### To match searches with bookings

In [2]:
import bz2
files_path = "/home/dsc/Repositories/Master-in-Data-Science/Data/"

##### Loading and formatting searches

In [10]:
searches_file = bz2.BZ2File(files_path+"searches.csv.bz2")
searches_df = pd.read_csv(searches_file, 
                          sep='^',
                          nrows=1e5,
                          usecols=['Origin','Destination','Seg1Date'],
                          parse_dates=['Seg1Date'])
searches_df['Is_Booked'] = 0
searches_df.reset_index(inplace=True)
bookings_file = bz2.BZ2File(files_path+"bookings.csv.bz2")
bookings_chk = pd.read_csv(bookings_file, sep='^',chunksize=1e5,usecols=['dep_port','arr_port','brd_time           ','pax'])

##### Loading and formatting bookings

In [11]:
n=0
for chunk in bookings_chk:
    n += 1
    if n>125:
        break
        
    # Discarding blanks, negatives bookings anf garbage rows
    chunk.dropna(inplace=True)
    chunk = chunk[chunk['pax'] >=1]
    chunk.drop('pax', axis=1, inplace=True)
    chunk = chunk[chunk['brd_time           '] != '3']

    # Formatting bookings dataframe to match searches dataframe
    chunk['Origin'] = chunk['dep_port'].apply(lambda s: s.strip())
    chunk.drop('dep_port', axis=1, inplace=True)

    chunk['Destination'] = chunk['arr_port'].apply(lambda s: s.strip())
    chunk.drop('arr_port', axis=1, inplace=True)

    chunk['Seg1Date'] = chunk['brd_time           '].apply(lambda d: pd.Timestamp(pd.to_datetime(d).date()))
    chunk.drop('brd_time           ', axis=1, inplace=True)

    # Adding requested column "if search = booking"
    chunk['Is_Booked_Chunk'] = 1
    
    # Obtaining searches with bookings
    searches_df = pd.merge(searches_df,chunk,on=['Origin','Destination','Seg1Date'],how="left").drop_duplicates()
    searches_df['Is_Booked_Chunk'].fillna(0,inplace=True)
    searches_df['Is_Booked'] = searches_df['Is_Booked_Chunk'].where(searches_df['Is_Booked_Chunk']==1,other=searches_df['Is_Booked'])
    searches_df.drop('Is_Booked_Chunk', axis=1, inplace=True)
    print n, len(searches_df), searches_df['Is_Booked'].sum()

        

   index Origin Destination   Seg1Date  Is_Booked  Is_Booked_Chunk
0      0    TXL         AUH 2013-01-26          0              NaN

1364.0
100000
   index Origin Destination   Seg1Date  Is_Booked  Is_Booked_Chunk
0      0    TXL         AUH 2013-01-26          0              NaN

2621.0
100000
   index Origin Destination   Seg1Date  Is_Booked  Is_Booked_Chunk
0      0    TXL         AUH 2013-01-26          0              NaN

3723.0
100000
   index Origin Destination   Seg1Date  Is_Booked  Is_Booked_Chunk
0      0    TXL         AUH 2013-01-26          0              NaN

4708.0
100000
   index Origin Destination   Seg1Date  Is_Booked  Is_Booked_Chunk
0      0    TXL         AUH 2013-01-26          0              NaN

5291.0
100000
   index Origin Destination   Seg1Date  Is_Booked  Is_Booked_Chunk
0      0    TXL         AUH 2013-01-26          0              NaN

5729.0
100000
   index Origin Destination   Seg1Date  Is_Booked  Is_Booked_Chunk
0      0    TXL         AUH 2013-01-26 

##### Writing new searches csv file

In [15]:
# Obtaining searches with bookings
new_searches_file = files_path+"new_searches.csv.bz2"
searches_df.to_csv(new_searches_file,sep='^',mode='a',compression='bz2')


In [ ]:
merged_df.head(1)